In [1]:

import torch
import torch.optim as optim
import torch.nn as nn
from parse_data import get_data, get_modified_values, get_binary_values, make_data_scalar
import numpy as np
import random
from data_gen import Datagen
from recognition import Recognition
from generator import Generator
from evaluation import evaluate_model, bin_plot
from time_recognition import TimeRecognition

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=None
print("Using device: ", device)

import torch
print(torch.__version__)
 

Using device:  cpu
2.5.1


In [2]:
gen = Datagen(device)

x, y, x_1 = gen.get_generated_data(seq_len=2)

print("x", x[0])
print("y", y[0])
print("x_1", x_1[0])

/home/kentagent/Documents/code/tDLGM/tdlgm_modifications/attention/data_gen.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]


x tensor([[0.],
        [0.]])
y tensor([1.])
x_1 tensor([[0.0000],
        [0.9974]])


In [3]:
#### import random

# Hyperparameters
sequence_length = [2*i for i in range(4,16)] # 2-20 increments of two
hidden_layers = [1,2] # 1 and 2
hidden_1 = [2**i for i in range(2,7)] # 2^4 to 2^9
hidden_2 =[2**i for i in range(5,10)] # 2^2 to 2^5
variance = [0.001, 0.01, 0.005, 0.05]
lr = [1/10**i for i in range(3,10)] # stop at 0.005
data_probability = [i/5 for i in range(1,6)]
regularization = [1/i for i in range(3,15)]
for i in range(3):
    regularization.append(0)

epochs = 1
optimizer = [optim.Adam, optim.SGD]

options = []

for seq_len in sequence_length:
    for layers in hidden_layers:
        for h1 in hidden_1:
            for h2 in hidden_2:
                for l in lr:
                    for v in variance:
                        for p in data_probability:
                            for r in regularization:
                                entry = {}
                                entry["seq_len"] = seq_len
                                entry["layers"] = layers
                                entry["latent"] = h1
                                entry["hidden"] = h2
                                entry["l"] = l
                                entry["variance"] = v
                                entry["data_prob"] = p
                                entry["regularization"] = r
                                options.append(entry)
                
                                         
random.shuffle(options)    

input_dim = 1
nn_size = [10,10]
state_dim = 2
batch_sizes = [4,3,2]
latent_dim = 2
output_dim = 1
seq_len=2

In [4]:
import torch

tensor = torch.rand(100, 24, 2)  # Example tensor
last_entry = tensor[:, -1, :]
print(last_entry.shape)

torch.Size([100, 2])


In [ ]:


import torch.utils.data as data
from itertools import chain
import torch.nn.functional as F

def loss(x, x_hat, mean, R, mean_state, R_state, device=None, seq_len=1):

    mse = nn.MSELoss().to(device)
    l = F.binary_cross_entropy(x_hat, x, reduction='sum')
    amount = mean[0].size()[0]*mean[0].size()[1]
    for m, r in zip(mean, R):

        C = r @ r.transpose(-2,-1)
        det = C.det()
        l += 0.5 * torch.sum(m.pow(2).sum(-1)
                             + C.diagonal(dim1=-2,dim2=-1).sum(-1)
                            -det.log()  -1)/amount

    amount = len(mean_state)*mean_state[0].size()[0]
    for m_l, r_l in zip(mean_state, R_state):
        for m, r in zip(m_l, r_l):

            C = r @ r.transpose(-1, 0)
            det = C.det()
            if det <= 0: # TODO, is there a better solution to this?
                continue
            

            l += 0.5* (
            torch.sum(torch.sum(m.pow(2).sum(-1)) + C.diagonal(dim1=-2, dim2=-1).sum(-1) - det.log() - 1)
            )/amount


    return l




best_model = None
best_score = 10000000000000000
batch_size = 100
best_history= [0,0,0,0,0,0]
for entry in options:

    x_d, y_d, x_1_d = gen.get_generated_data(entry["seq_len"], entry["variance"], entry["data_prob"])
    x_t, y_t, x_t_1 = gen.get_true_data(entry["seq_len"])
    x_val, y_val, x_val_1 = gen.get_test_data(entry["seq_len"])



    
    model_t = TimeRecognition(input_dim=input_dim,
                              network_size=nn_size,
                              batches=batch_sizes,
                              seq_len=seq_len,
                              state_dim=state_dim)

    model_g = Generator(state_dim=state_dim,
                        network_size=nn_size,
                        latent_dim=latent_dim,
                        layers=layers,
                        num_heads=2,
                        output_dim=output_dim,
                        seq_len=seq_len,
                        activation_function=nn.Sigmoid,
                        device=device)
    
    model_r = Recognition(input_dim=input_dim,
                          latent_dim=latent_dim,
                          layers=layers)

    #evaluate_model(model_g,model_r, model_t, x_t, y_t, x_t_1,x_val,y_val, x_val_1, entry)


    loader = data.DataLoader(data.TensorDataset(x_d, y_d, x_1_d), batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(chain(model_r.parameters(), model_g.parameters(), model_t.parameters()), lr=entry["l"])
    #optimizer = optim.Adam(model_r.parameters())
    history = []
    bce = nn.BCELoss().to(device)
    for e in range(epochs):
        model_g.train()
        model_r.train()
        model_t.train()


        for x, y, x_1 in loader:

            x.to(device)
            y.to(device)
            if x.size()[0] < batch_size:
                continue
            if random.random() < 0.5:
                continue


            mean_state, log_var, state = model_t(x)
            mean, R, z = model_r(x_1[:,-1,:])
            model_g.set_xi(z)
            res = model_g(state)

            l = loss(y, res, mean, R, mean_state, log_var, device)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
                        

        
        if e % 10 != 0:
            continue
        
        count = 0
        sum_loss = [0, 0]
        for j in range(2):
            for x, y, x_1 in loader:
                model_g.eval()
                model_t.eval()
                model_r.eval()
                model_g.make_xi()
                with torch.no_grad():
                    mean_state, log_var, state = model_t(x)
                    mean, R, z = model_r(x_1[:,-1,:])
                    model_g.set_xi(z)
                    res = model_g(state)

                    l = loss(y, res, mean, R, mean_state, log_var, device)
                    res = []
                    
                    sum_loss[j] += l.item()
                    count += 1
                    
        
        
        sum_loss[0] /= count
     
        
        history.append([e, sum_loss[0], sum_loss[1]])
        print(history[-1])

        if len(history) > 15:
            #if no real improvements are being done stop the training. 
            # but keep doing the training if the results without correctly feeding values get better
            if abs(history[-15][1] - history[-1][1]) < 0.0001:
                break
        

    if history[-1][1] < best_score:
        print("New best model:\nNew loss: ", history[-1], "\nOld loss:", best_history[-1], "\nHistory:" , history[-10:])
        best_model = model_g
        best_history = history
        best_score = history[-1][1]
        best_config = entry
        with torch.no_grad():
            evaluate_model(best_model,model_r, model_t, x_t, y_t, x_t_1,x_val,y_val, x_val_1, entry)
    else:
        with torch.no_grad():
            evaluate_model(model_g,model_r, model_t, x_t, y_t, x_t_1,x_val,y_val, x_val_1, entry)
        print("Old model still stands:\nCurrent loss: ", history[-1], "\nBest loss:", best_history[-1])
    

/home/kentagent/Documents/code/tDLGM/tdlgm_modifications/attention/data_gen.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]


[0, 46.404073616126915, 7149.893775939941]
New best model:
New loss:  [0, 46.404073616126915, 7149.893775939941] 
Old loss: 0 
History: [[0, 46.404073616126915, 7149.893775939941]]


/home/kentagent/Documents/code/tDLGM/tdlgm_modifications/attention/data_gen.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]


In [ ]:
a = torch.zeros(10,5,1)

In [ ]:
a[:,-1,:].size()

In [ ]:
b = torch.zeros(30,1)
c = torch.zeros(1,1)


In [ ]:
torch.cat((b[1:],c))

In [ ]:
23